In [147]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd

In [213]:
#riot games api key
api_key = 'RGAPI-eccf63ee-33e0-42f6-a7a0-977beed8833b'

#set region, init lol watcher obj
region = 'na1'
lol_watcher = LolWatcher(api_key)
me = 'Tape'

In [214]:
lol_watcher.summoner.by_name(region, me)

{'id': '_U9EHzuUdBPEv16p44wnZ1nkuHalvKim1nqZNky9_BEKdKY',
 'accountId': 'Y6I9cQlE7hh1bXisAJEsoocfA6uLJI7Wy8FAKkGXbdTvfQ',
 'puuid': 'slo7o0JekUTfZEnoL3XdBjie0yX7goflVGIpFYNHdzZ1ES78SfbyakIPIW4ffgoXAkafrBf0oVA2Lg',
 'name': 'Tape',
 'profileIconId': 5659,
 'revisionDate': 1674691198692,
 'summonerLevel': 395}

In [215]:
#return challengers
challengers = lol_watcher.league.challenger_by_queue(region, 'RANKED_SOLO_5x5')

#return grandmasters
gms = lol_watcher.league.grandmaster_by_queue(region, 'RANKED_SOLO_5x5')

#return masters
masters = lol_watcher.league.masters_by_queue(region, 'RANKED_SOLO_5x5')

In [216]:
#list of the above objects
all_top_players = [challengers, gms, masters]

#loop through and concat all summoner Ids
summoner_ids = []
for division in all_top_players:
    for entry in division['entries']:
        summoner_ids.append(list(entry.values())[0])

In [217]:
#now we have all summoner IDs in a list
#because my key is a development key, limited to 100 requests every 2 minutes
#i want to make a lot more queries than that

In [218]:
#map summid to puuid
summid_to_puuid = {}
for summoner in summoner_ids:
    summid_to_puuid[summoner] =  lol_watcher.summoner.by_id(region, summoner)['puuid']

In [ ]:
#create dict to hold champion masteries for each summoner by puuid
mastery_dict = {}

#fill dict
for summoner in summoner_ids:
    #make request for champion masteries, store in variable
    masteries = lol_watcher.champion_mastery.by_summoner(region, summoner)
    #loop through, adding high mastery champs to list
    puuid = summid_to_puuid[summoner]
    mastery_dict[puuid] = []
    for mastery in masteries:
        if mastery.get('championPoints') > 100000:
            mastery_dict[puuid].append(mastery.get('championId'))

In [ ]:
#okay, now we could iterate over each summoner and grab matches. However, we could likely
#improve runtime by checking for each summoner in our list to see if they were in the game

#create list to store dict objects
data_rows = []
            
#store set of matches already looked through
matches_scanned = set()

In [ ]:
#list of features we want to record
features = ['puuid',
            'championId',
            'item0',
            'item1',
            'item2',
            'item3',
            'item4',
            'item5',
            'item6',
            'kills',
            'deaths',
            'assists',
            'totalDamageDealtToChampions',
            'role',
            'teamPosition',
            'teamId',
            'gameEndedInEarlySurrender',
            'win']

# TODO: add error 503 handling
for key, value in mastery_dict.items():
    #store matchlist for each puuid
    match_list = lol_watcher.match.matchlist_by_puuid(region, key, count = 50)
    for match in match_list:
        if match not in matches_scanned:
            #store match data in variable
            match_data = lol_watcher.match.by_id(region, match)
            #store participant information in variable to iterate over (list of dicts) if classic game
            if match_data['info']['gameMode'] == 'CLASSIC':
                player_info = match_data['info']['participants']
                #create dict of champs on team1, team2
                champions_in_game = {}
                champions_in_game[100] = []
                champions_in_game[200] = []
                for player in player_info:
                    #add champ played to dict
                    champions_in_game[player['teamId']].append(player['championId'])
                    #check to see if player in our list of masters+ players
                    if player['puuid'] in mastery_dict.keys(): 
                        #check to see if player on a high mastery champ
                        if player['championId'] in mastery_dict[player['puuid']]:
                            #get player data, store in dictionary
                            player_data = {}
                            for feature in features:
                                player_data[feature] = player[feature]
                            player_data['match_id'] = match
                            player_data['champions_in_game'] = champions_in_game
                            #append dictionary to list
                            data_rows.append(player_data)
                            
                #print out to watch progress
                #print('champion ID: ',player_data['championId'],', win:',player_data['win'])
                #print('champs in game: ',player_data['champions_in_game'])
                            
            #append match_id to matches_scanned set
            matches_scanned.add(match)



In [ ]:
#do we have info for every champion?
champs_logged = set()
for row in data_rows:
    champs_logged.add(row['championId'])
len(champs_logged)

In [145]:
#we dooooooooo
#construct a row for each entry in the data_rows object to go into our dataframe

In [196]:
df = pd.DataFrame.from_dict(data_rows)

In [200]:
#drop where teamPosition empty
df = df[df['teamPosition'] != '']
#drop where game ended in early surrender
df = df[df['gameEndedInEarlySurrender'] == False]

In [208]:
df[['teamId','champions_in_game']]

,teamId,champions_in_game
0,100,"{100: [777, 147, 555], 200: [897, 360]}"
1,100,"{100: [777, 147, 555], 200: [897, 360]}"
2,200,"{100: [777, 147, 555], 200: [897, 360]}"
3,200,"{100: [777, 147, 555], 200: [897, 360]}"
4,200,"{100: [777, 147, 555], 200: [897, 360]}"
...,...,...
47383,100,"{100: [14, 131, 222], 200: [57, 238, 145, 101]}"
47384,100,"{100: [14, 131, 222], 200: [57, 238, 145, 101]}"
47385,200,"{100: [14, 131, 222], 200: [57, 238, 145, 101]}"
47386,100,"{100: [27, 104, 119, 517], 200: [13, 91, 110, ..."


In [202]:
#lets construct columns from the teamId and champions_in_game column

#new column, list of champions on team, and drop self from list of teammates
df['teammates_championId'] = df.apply(lambda x: x['champions_in_game'].get(x['teamId']).remove(x['championId']), axis=1)

#new column, list of enemy champions
opposite_team_dict = {100:200, 200:100}
df['opposite_team_id'] = df['teamId'].map(opposite_team_dict)
df['enemies_championId'] = df.apply(lambda x: x['champions_in_game'].get(x['opposite_team_id']), axis=1)

ValueError: list.remove(x): x not in list

In [212]:
df

,puuid,championId,item0,item1,item2,item3,item4,item5,item6,kills,deaths,assists,totalDamageDealtToChampions,role,teamPosition,teamId,gameEndedInEarlySurrender,win,match_id,champions_in_game
0,eqtbPmaad6RjSnBajLdcLjGWqweee05KMK86DLSPmaMAMc...,164,2420,3074,3133,3047,6632,1029,3363,0,6,10,16240,SOLO,TOP,100,False,True,NA1_4557967961,"{100: [777, 147, 555], 200: [897, 360]}"
1,zZdvTYj53X_08p7eXfaq5RxU62xo1PKB2wFZvYyoLJUX00...,76,3157,3020,3041,6655,3135,1058,3364,14,3,10,39386,NONE,JUNGLE,100,False,True,NA1_4557967961,"{100: [777, 147, 555], 200: [897, 360]}"
2,5GMEtXdDnmxWkyYN4Ko_lqraFDsuu4_aSM8KPlfSMiVmi2...,203,0,3047,6671,1038,1037,6676,3364,4,8,4,9701,NONE,JUNGLE,200,False,False,NA1_4557967961,"{100: [777, 147, 555], 200: [897, 360]}"
3,Xj-7Erni2nIejNpWVh3Aaiymsm93y6UuyRWESIF5BsX8-n...,41,3508,6693,6676,2422,1018,6675,3364,3,10,7,27084,DUO,MIDDLE,200,False,False,NA1_4557967961,"{100: [777, 147, 555], 200: [897, 360]}"
4,FAIHMoaHkLrAppXntUsNrRqXQvaUh2cNuWmXTf7h3MqaTu...,12,0,3117,6667,0,3157,3860,3364,3,5,12,5523,SUPPORT,UTILITY,200,False,False,NA1_4557967961,"{100: [777, 147, 555], 200: [897, 360]}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47383,M32_ErMOxs3UuX1rZH0f5bsWsqHWmXOTunZC5uOu3ojs6O...,35,4637,6653,3158,1052,0,0,3364,6,5,4,16763,NONE,JUNGLE,100,False,False,NA1_4545430017,"{100: [14, 131, 222], 200: [57, 238, 145, 101]}"
47384,Hub7jhQJ0QFP3AcA-WzunhJQ4a9qmmzlp7sAtZDIekyjb4...,235,3179,3864,2055,3009,6692,0,3364,2,7,4,12813,SUPPORT,UTILITY,100,False,False,NA1_4545430017,"{100: [14, 131, 222], 200: [57, 238, 145, 101]}"
47385,j__vPtfS5cFRe2u4SGtDExVT4lb-JiRrNMsnJ1u7tiX_dj...,80,2033,0,6632,3158,3155,3071,3364,5,2,9,26157,SOLO,TOP,200,False,True,NA1_4545430017,"{100: [14, 131, 222], 200: [57, 238, 145, 101]}"
47386,Sk1xjH-Ov26UOAgwzHJVr4FEBHo4XOvverL_MBJAMye51T...,55,3152,3020,2421,3153,3115,3191,3363,12,11,7,29244,CARRY,MIDDLE,100,False,False,NA1_4545416147,"{100: [27, 104, 119, 517], 200: [13, 91, 110, ..."


In [204]:
x['champions_in_game'].get(x['teamId'])

[777, 147, 555]

In [ ]:
#split list into individual columns
pd.DataFrame(df2["teams"].to_list(), columns=['team1', 'team2'])

puuid                          eqtbPmaad6RjSnBajLdcLjGWqweee05KMK86DLSPmaMAMc...
championId                                                                   164
item0                                                                       2420
item1                                                                       3074
item2                                                                       3133
item3                                                                       3047
item4                                                                       6632
item5                                                                       1029
item6                                                                       3363
kills                                                                          0
deaths                                                                         6
assists                                                                       10
totalDamageDealtToChampions 

In [ ]:
opposite_team_dict = {100:200, 200:100}
df['opposite_team_id'] = df['teamId'].map(opposite_team_dict)